In [ ]:
!wget -q http://go.criteo.net/criteo-research-uplift-v2.1.csv.gz

In [ ]:
!gzip -d /content/criteo-research-uplift-v2.1.csv.gz

gzip: /content/criteo-research-uplift-v2.1.csv already exists; do you wish to overwrite (y or n)? 

In [ ]:
! pip install -q scikit-uplift causalml catboost

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Импортируем S- и T- learner`ы  (SoloModel и TwoModels) из библиотеки scikit-uplift (sklift)
from sklift.models import SoloModel, TwoModels

# Импортируем S- и T-learner`ы (BaseSClassifier и BaseTClassifier) из библиотеки CausalML
# Нам нужны именно Classifier, так как мы будем решать задачу классификации
# Аналогичные реализации (Regressor) есть и для задач регрессии
from causalml.inference.meta import BaseSClassifier, BaseTClassifier


# В качестве классификатора воспользуемся моделью градиентного бустинга от Яндекса (CatBoost)
from catboost import CatBoostClassifier

# Прочитаем файл с данными
df = pd.read_csv("/content/criteo-research-uplift-v2.1.csv")
df.head(2)

In [ ]:
df['treatment'].value_counts()

In [ ]:
df.info()

In [ ]:
list(df.columns)

In [ ]:
#todo(alex) here
# Определим колонки с факторами, тритментом и таргетом
feature_cols = ['f0',
                'f1',
                'f2',
                'f3',
                'f4',
                'f5',
                'f6',
                'f7',
                'f8',
                'f9',
                'f10',
                'f11']

target_col = 'conversion'
treatment_col = 'treatment'

In [ ]:
df_discount_train, df_discount_test = train_test_split(
    df,
    stratify=df[[treatment_col, target_col]],
    random_state=13,
    test_size=0.3
)

In [ ]:
# Возьмем функцию для оценки qini-curve с прошлого занятия
def qini_df(df, title='train', figsize=(5, 3)):
    # Отранжируем выборку по значению uplift в убывающем порядке
    ranked = df.sort_values("uplift_score", ascending=False)

    N_c = sum(ranked['target_class'] <= 1)
    N_t = sum(ranked['target_class'] >= 2)

    # Посчитаем в отсортированном датафрейме основные показатели, которые используются при расчете qini
    ranked['n_c1'] = 0
    ranked['n_t1'] = 0
    ranked.loc[ranked.target_class == 1,'n_c1'] = 1
    ranked.loc[ranked.target_class == 3,'n_t1'] = 1
    ranked['n_c1/nc'] = ranked.n_c1.cumsum() / N_c
    ranked['n_t1/nt'] = ranked.n_t1.cumsum() / N_t

    # Посчитаем qini curve и рандомную прямую под ней
    ranked['uplift'] = round(ranked['n_t1/nt'] - ranked['n_c1/nc'],5)
    # Добавим случайную кривую
    ranked['random_uplift'] = round(ranked["uplift_score"].rank(pct=True, ascending=False) * ranked['uplift'].iloc[-1],5)

    ranked["n"] = ranked["uplift_score"].rank(pct=True, ascending=False)

    # Немного кода для визуализации
    fig = plt.figure(figsize=figsize)
    plt.plot(ranked['n'], ranked['uplift'], color='r', label='Model')
    plt.plot(ranked['n'], ranked['random_uplift'], color='b', label='RandomModel')
    plt.legend()
    plt.title('Qini-curve for {} samples'.format(title))
    plt.show()
    quni_score = (ranked['uplift'] - ranked['random_uplift']).sum()
    print('Qini score: {:.3f}'.format(quni_score))

In [ ]:
# Давайте по данным построим S-learner

# Создадим базовый S-learner
s_learner = BaseSClassifier(learner=CatBoostClassifier(random_seed=13, verbose=0))

# Для обучения нам нужны датафрем с факторами, колонка с фактом воздействия
s_learner.fit(X=df_discount_train[feature_cols],
              treatment=df_discount_train[treatment_col],
              y=df_discount_train[target_col])

# Сделаем предсказание uplift-эффекта на обучающей выборке
uplift_vals = s_learner.predict(np.array(df_discount_train[feature_cols].values.copy()))
df_discount_train['uplift_score'] = uplift_vals

# Сделаем предсказание uplift-эффекта на тестовой выборке
uplift_vals = s_learner.predict(np.array(df_discount_test[feature_cols].values.copy()))
df_discount_test['uplift_score'] = uplift_vals

# Мы получили какие-то значения рамках решения задачи классификации, давайте посмотрим на qini score
qini_df(df_discount_train, title='train')
qini_df(df_discount_test, title='test')

In [ ]:
# Давайте по данным построим T-learner

# Создадим базовый T-learner
t_learner = BaseTClassifier(learner=CatBoostClassifier(random_seed=13, verbose=0))

# Для обучения нам нужны датафрем с факторами, колонка с фактом воздействия
t_learner.fit(X=df_discount_train[feature_cols],
              treatment=df_discount_train[treatment_col],
              y=df_discount_train[target_col])

# Сделаем предсказание uplift-эффекта на обучающей выборке
uplift_vals = t_learner.predict(np.array(df_discount_train[feature_cols].values.copy()))
df_discount_train['uplift_score'] = uplift_vals

# Сделаем предсказание uplift-эффекта на тестовой выборке
uplift_vals = t_learner.predict(np.array(df_discount_test[feature_cols].values.copy()))
df_discount_test['uplift_score'] = uplift_vals

# Мы получили какие-то значения рамках решения задачи классификации, давайте посмотрим на qini score
qini_df(df_discount_train, title='train')
qini_df(df_discount_test, title='test')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression

# Импортируем X- и R-learner`ы (BaseXClassifier и BaseRClassifier) из библиотеки CausalML
# Нам нужны именно Classifier, так как мы будем решать задачу классификации
# Аналогичные реализации (Regressor) есть и для задач регрессии
from causalml.inference.meta import BaseXClassifier, BaseRClassifier
from catboost import CatBoostClassifier, CatBoostRegressor


In [ ]:
# Давайте по данным построим X-learner

# Создадим базовый X-learner
x_learner = BaseXClassifier(
    outcome_learner=CatBoostClassifier(depth=5, random_seed=13, verbose=0),
    effect_learner=LinearRegression()
)

# Для обучения нам нужны датафрем с факторами, колонка с фактом воздействия
x_learner.fit(
    X=df_discount_train[feature_cols],
    treatment=df_discount_train[treatment_col],
    y=df_discount_train[target_col]
)

# Сделаем предсказание uplift-эффекта на обучающей выборке
uplift_vals = x_learner.predict(np.array(df_discount_train[feature_cols].values.copy()))
df_discount_train['uplift_score'] = uplift_vals

# Сделаем предсказание uplift-эффекта на тестовой выборке
uplift_vals = x_learner.predict(np.array(df_discount_test[feature_cols].values.copy()))
df_discount_test['uplift_score'] = uplift_vals

# Мы получили какие-то значения рамках решения задачи классификации, давайте посмотрим на qini score
qini_df(df_discount_train, title='train')
qini_df(df_discount_test, title='test')

In [ ]:
# Давайте по данным построим R-learner

# Создадим базовый R-learner
r_learner = BaseRClassifier(
    outcome_learner=CatBoostClassifier(depth=5, random_seed=13, verbose=0),
    effect_learner=LinearRegression(),
    random_state=42
)

# Для обучения нам нужны датафрем с факторами, колонка с фактом воздействия
r_learner.fit(X=df_discount_train[feature_cols],
              treatment=df_discount_train[treatment_col],
              y=df_discount_train[target_col])

# Сделаем предсказание uplift-эффекта на обучающей выборке
uplift_vals = r_learner.predict(np.array(df_discount_train[feature_cols].values.copy()))
df_discount_train['uplift_score'] = uplift_vals

# Сделаем предсказание uplift-эффекта на тестовой выборке
uplift_vals = r_learner.predict(np.array(df_discount_test[feature_cols].values.copy()))
df_discount_test['uplift_score'] = uplift_vals

# Мы получили какие-то значения рамках решения задачи классификации, давайте посмотрим на qini score
qini_df(df_discount_train, title='train')
qini_df(df_discount_test, title='test')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

from sklearn.model_selection import train_test_split

# Продолжаем решать нашу постановку как задачу классификации - для этого возьмем одно Uplift-дерево и случайный лес
# помним, что основное отличие - это функция разбиения в листе
from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot

In [ ]:
# Создаем простое дерево.
uplift_tree = UpliftTreeClassifier(
    max_depth=5, min_samples_leaf=200,
    min_samples_treatment=50,
    n_reg=100, evaluationFunction='ED',
    control_name='0', random_state=42
)

# Для обучения нам нужны датафрем с факторами, колонка с фактом воздействия
# Обратите внимание, что для использования деревьев из CausalML нам необходимо преобразовать фактор воздействия в строку
uplift_tree.fit(
    df_discount_train[feature_cols].values,
    treatment=df_discount_train[treatment_col].apply(str).values,
    y=df_discount_train[target_col].values
)

# Сделаем предсказание uplift-эффекта на обучающей выборке
# Функция predict у деревьев возвращает uplift-эффекты для каждой группы воздействия
# Нас интересует только наличие воздействия (столбец под индексом 1)
uplift_vals = uplift_tree.predict(np.array(df_discount_train[feature_cols].values.copy()))[:, 1]
df_discount_train['uplift_score'] = uplift_vals

# Сделаем предсказание uplift-эффекта на тестовой выборке
# Функция predict у деревьев возвращает uplift-эффекты для каждой группы воздействия
# Нас интересует только наличие воздействия (столбец под индексом 1)
uplift_vals = uplift_tree.predict(np.array(df_discount_test[feature_cols].values.copy()))[:, 1]
df_discount_test['uplift_score'] = uplift_vals

# Мы получили какие-то значения рамках решения задачи классификации, давайте посмотрим на qini score
qini_df(df_discount_train, title='train')
qini_df(df_discount_test, title='test')

In [ ]:
# Давайте визуализируем наше простейшее дерево
from IPython.display import Image
# Вызываем функцию для визуализации:
graph = uplift_tree_plot(uplift_tree.fitted_uplift_tree, feature_cols)
Image(graph.create_png())

In [ ]:
# Создаем простое дерево.
uplift_forest = UpliftRandomForestClassifier(
    n_estimators=100, max_depth=5, min_samples_leaf=100,
    min_samples_treatment=50,
    n_reg=100, evaluationFunction='ED',
    control_name='0', random_state=42
)

# Для обучения нам нужны датафрем с факторами, колонка с фактом воздействия
# Обратите внимание, что для использования деревьев из CausalML нам необходимо преобразовать фактор воздействия в строку
uplift_forest.fit(
    df_discount_train[feature_cols].values,
    treatment=df_discount_train[treatment_col].apply(str).values,
    y=df_discount_train[target_col].values
)

# Сделаем предсказание uplift-эффекта на обучающей выборке
# Функция predict у деревьев возвращает uplift-эффекты для каждой группы воздействия
# Нас интересует только наличие воздействия (столбец под индексом 1)
uplift_vals = uplift_forest.predict(np.array(df_discount_train[feature_cols].values.copy()))
df_discount_train['uplift_score'] = uplift_vals

# Сделаем предсказание uplift-эффекта на тестовой выборке
# Функция predict у деревьев возвращает uplift-эффекты для каждой группы воздействия
# Нас интересует только наличие воздействия (столбец под индексом 1)
uplift_vals = uplift_forest.predict(np.array(df_discount_test[feature_cols].values.copy()))
df_discount_test['uplift_score'] = uplift_vals

# Мы получили какие-то значения рамках решения задачи классификации, давайте посмотрим на qini score
qini_df(df_discount_train, title='train')
qini_df(df_discount_test, title='test')